<a href="https://colab.research.google.com/github/0AlphaZero0/IBM-Coursera-AppliedDataScienceCapstone/blob/master/IBM_Capstone7_DASH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label':'All sites','value':'OPT5'},
                                        {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                        {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                        {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                        {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}
                                        ],
                                    value='OPT5'                                        
                                ),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload,max_payload],
                                     marks={
                                        0: '0',
                                        2500: '2500',
                                        5000: '5000',
                                        7500: '7500',
                                        10000: '10000'
                                    },
                                ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(
                                    dcc.Graph(id='success-payload-scatter-chart')
                                ),
                            ]
                        )

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart',component_property='figure'),
    Input(component_id='site-dropdown',component_property='value')
)
def update_pie(site_value):
    if site_value=="OPT5":
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site')
        return fig
    else:
        filtereddf = spacex_df[spacex_df['Launch Site']==site_value].groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(
            filtereddf,
            values='class count',
            names='class'
        )
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id="payload-slider", component_property="value")
    ]    
)
def update_scatterplot(site,payload):
    if site=="OPT5":
        fig = px.scatter(
            spacex_df[(spacex_df['Payload Mass (kg)']>=payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])],
            x='Payload Mass (kg)',
            y='class',
            color="Booster Version Category"
        )
        return fig
    else:
        site_filter_df=spacex_df[spacex_df['Launch Site']==site]
        filtereddf=site_filter_df[(site_filter_df['Payload Mass (kg)']>=payload[0]) & (site_filter_df['Payload Mass (kg)']<=payload[1])]
        fig = px.scatter(
            filtereddf,
            x='Payload Mass (kg)',
            y='class',
            color="Booster Version Category"
        )
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server()
